# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [4]:
import syft as sy

In [6]:
hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [7]:
bob = sy.VirtualWorker(hook, id="bob")

In [9]:
bob._objects

{}

In [10]:
x = th.tensor([1,2,3,4,5])

In [11]:
x = x.send(bob)

In [12]:
bob._objects

{63979468430: tensor([1, 2, 3, 4, 5])}

In [13]:
x

(Wrapper)>[PointerTensor | me:32772398522 -> bob:63979468430]

In [14]:
x.location

<VirtualWorker id:bob #objects:1>

In [15]:
x.location == bob 

True

In [16]:
x.id_at_location

63979468430

In [17]:
x.id

32772398522

In [18]:
x.owner

<VirtualWorker id:me #objects:0>

In [19]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [20]:
x

(Wrapper)>[PointerTensor | me:32772398522 -> bob:63979468430]

In [21]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [22]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [23]:
# try this project here!

In [8]:
alice = sy.VirtualWorker(hook, id="alice")


In [25]:
x = th.tensor([1,2,3,4,5])


In [26]:
x_ptr = x.send(bob, alice)


In [27]:
x_ptr

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:11502041420 -> bob:98001828985]
	-> (Wrapper)>[PointerTensor | me:7931755936 -> alice:3196144960]

In [28]:
x_ptr.child

MultiPointerTensor>{'bob': (Wrapper)>[PointerTensor | me:11502041420 -> bob:98001828985], 'alice': (Wrapper)>[PointerTensor | me:7931755936 -> alice:3196144960]}

In [29]:
x_ptr.get()


[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [30]:
# x_ptr.get(sum_results=True)


# Lesson: Introducing Remote Arithmetic

In [31]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [32]:
x

(Wrapper)>[PointerTensor | me:94481966350 -> bob:13933825495]

In [33]:
y

(Wrapper)>[PointerTensor | me:18585889577 -> bob:63824796244]

In [34]:
z = x + y

In [35]:
z

(Wrapper)>[PointerTensor | me:36721247795 -> bob:70655015436]

In [36]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [37]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:74255232610 -> bob:59718773413]

In [38]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [39]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [40]:
z = (x + y).sum()

In [41]:
z.backward()

(Wrapper)>[PointerTensor | me:93221352631 -> bob:69533327757]

In [42]:
x = x.get()

In [43]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [44]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [45]:
# try this project here!

In [46]:
input = th.tensor([[1.,1],[0,1,],[1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)


In [47]:
weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob)


In [48]:
pred = input.mm(weights)

In [49]:
loss = ((pred - target)**2).sum()

In [50]:
loss.backward()


(Wrapper)>[PointerTensor | me:84642302014 -> bob:93516720911]

In [51]:
weights.data.sub_(weights.grad * 0.1)

(Wrapper)>[PointerTensor | me:74204901000 -> bob:47844362432]::data

In [52]:
weights.grad *= 0


In [53]:
print(loss.get().data)


tensor(2.)


In [54]:
for i in range(10):

    pred = input.mm(weights)

    loss = ((pred - target)**2).sum()

    loss.backward()

    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0

    print(loss.get().data)

tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)
tensor(0.0058)


# Lesson: Garbage Collection and Common Errors


In [55]:
bob = bob.clear_objects()

In [56]:
bob._objects

{}

In [57]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [58]:
bob._objects

{25146693368: tensor([1, 2, 3, 4, 5])}

In [59]:
del x

In [60]:
bob._objects

{}

In [61]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [62]:
bob._objects

{69627668430: tensor([1, 2, 3, 4, 5])}

In [63]:
x = "asdf"

In [64]:
bob._objects

{}

In [65]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [66]:
x

(Wrapper)>[PointerTensor | me:8456297744 -> bob:72278673303]

In [67]:
bob._objects

{72278673303: tensor([1, 2, 3, 4, 5])}

In [68]:
x = "asdf"

In [69]:
bob._objects

{72278673303: tensor([1, 2, 3, 4, 5])}

In [70]:
del x

In [71]:
bob._objects

{72278673303: tensor([1, 2, 3, 4, 5])}

In [72]:
bob = bob.clear_objects()
bob._objects

{}

In [73]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [74]:
bob._objects

{22823078255: tensor([1, 2, 3, 4, 5])}

In [75]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [76]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:11797706736 -> bob:45534951053]
Tensor B: tensor([1, 1, 1, 1, 1])

In [77]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [78]:
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:bob #objects:2> while the other is on <VirtualWorker id:alice #objects:1>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [79]:
from torch import nn, optim

In [80]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [81]:
# A Toy Model
model = nn.Linear(2,1)

In [82]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [83]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(0.4935)
tensor(0.1276)
tensor(0.0675)
tensor(0.0421)
tensor(0.0269)
tensor(0.0172)
tensor(0.0110)
tensor(0.0071)
tensor(0.0045)
tensor(0.0029)
tensor(0.0019)
tensor(0.0012)
tensor(0.0008)
tensor(0.0005)
tensor(0.0003)
tensor(0.0002)
tensor(0.0001)
tensor(8.2486e-05)
tensor(5.2963e-05)
tensor(3.4029e-05)


In [84]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [85]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [86]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [87]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [88]:
train()

tensor(9.7322, requires_grad=True)
tensor(0.1170, requires_grad=True)
tensor(0.1932, requires_grad=True)
tensor(0.1600, requires_grad=True)
tensor(0.0581, requires_grad=True)
tensor(0.1073, requires_grad=True)
tensor(0.0352, requires_grad=True)
tensor(0.0700, requires_grad=True)
tensor(0.0230, requires_grad=True)
tensor(0.0462, requires_grad=True)
tensor(0.0154, requires_grad=True)
tensor(0.0308, requires_grad=True)
tensor(0.0105, requires_grad=True)
tensor(0.0208, requires_grad=True)
tensor(0.0073, requires_grad=True)
tensor(0.0142, requires_grad=True)
tensor(0.0052, requires_grad=True)
tensor(0.0098, requires_grad=True)
tensor(0.0037, requires_grad=True)
tensor(0.0069, requires_grad=True)
tensor(0.0027, requires_grad=True)
tensor(0.0048, requires_grad=True)
tensor(0.0019, requires_grad=True)
tensor(0.0034, requires_grad=True)
tensor(0.0014, requires_grad=True)
tensor(0.0025, requires_grad=True)
tensor(0.0011, requires_grad=True)
tensor(0.0018, requires_grad=True)
tensor(0.0008, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [9]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [10]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [11]:
x = x.send(alice)

In [12]:
bob._objects

{29368629401: tensor([1, 2, 3, 4, 5])}

In [13]:
alice._objects

{35994806220: (Wrapper)>[PointerTensor | alice:35994806220 -> bob:29368629401]}

In [14]:
y = x + x

In [15]:
y

(Wrapper)>[PointerTensor | me:49096168502 -> alice:12022328005]

In [16]:
bob._objects

{29368629401: tensor([1, 2, 3, 4, 5]),
 59669134576: tensor([ 2,  4,  6,  8, 10])}

In [17]:
alice._objects

{35994806220: (Wrapper)>[PointerTensor | alice:35994806220 -> bob:29368629401],
 12022328005: (Wrapper)>[PointerTensor | alice:12022328005 -> bob:59669134576]}

In [18]:
jon = sy.VirtualWorker(hook, id="jon")

In [19]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [20]:
bob._objects

{93424332916: tensor([1, 2, 3, 4, 5])}

In [21]:
alice._objects

{76506625026: (Wrapper)>[PointerTensor | alice:76506625026 -> bob:93424332916]}

In [22]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:76506625026 -> bob:93424332916]

In [23]:
bob._objects

{93424332916: tensor([1, 2, 3, 4, 5])}

In [24]:
alice._objects

{}

In [25]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [26]:
bob._objects

{}

In [27]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [28]:
bob._objects

{33218555321: tensor([1, 2, 3, 4, 5])}

In [29]:
alice._objects

{29923542573: (Wrapper)>[PointerTensor | alice:29923542573 -> bob:33218555321]}

In [30]:
del x

In [31]:
bob._objects

{}

In [32]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [33]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [34]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [35]:
bob._objects

{24003489268: tensor([1, 2, 3, 4, 5])}

In [36]:
alice._objects

{}

In [37]:
x.move(alice)

(Wrapper)>[PointerTensor | me:85114594734 -> alice:85114594734]

In [38]:
bob._objects

{}

In [39]:
alice._objects

{85114594734: tensor([1, 2, 3, 4, 5])}

In [40]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [41]:
bob._objects

{7680067717: tensor([1, 2, 3, 4, 5])}

In [42]:
alice._objects

{85114594734: tensor([1, 2, 3, 4, 5]),
 59105205247: (Wrapper)>[PointerTensor | alice:59105205247 -> bob:7680067717]}

In [44]:
x.remote_get()

InvalidTensorForRemoteGet: Tensor does not have attribute child. You remote get should be called on a chain of pointer tensors, instead you called it on tensor([1, 2, 3, 4, 5]).

In [45]:
bob._objects

{}

In [46]:
alice._objects

{85114594734: tensor([1, 2, 3, 4, 5]), 59105205247: tensor([1, 2, 3, 4, 5])}

In [47]:
x.move(bob)

(Wrapper)>[PointerTensor | me:64849812702 -> bob:64849812702]

In [48]:
x

(Wrapper)>[PointerTensor | me:64849812702 -> bob:64849812702]

In [49]:
bob._objects

{64849812702: tensor([1, 2, 3, 4, 5])}

In [50]:
alice._objects

{85114594734: tensor([1, 2, 3, 4, 5])}